In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
from torch.autograd import Variable 
from torchvision import transforms 
from data_loader import build_vocab 
from model import EncoderCNN, DecoderRNN
from model import ResNet, ResidualBlock
from attn_model import ResidualBlock, AttnEncoder, AttnDecoderRnn
from PIL import Image


embed_size = 256
hidden_size = 512 
feature_size = 256
num_layers = 1 
decoder_path = './models/attn/1object/decoder-10-150.pkl'
encoder_path = './models/attn/1object/encoder-10-150.pkl'


def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

def load_image(image_path, transform=None):
    image = Image.open(image_path).convert('RGB')
    image = image.resize([64, 64], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)
    
    return image

transform = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize((0.033, 0.032, 0.033), 
                         (0.027, 0.027, 0.027))])



vocab_path = './data/attn/vocab1.pkl'
with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)
len_vocab = vocab.idx

# Build Models
encoder = AttnEncoder(ResidualBlock, [3, 3, 3])
encoder.eval()  # evaluation mode (BN uses moving mean/variance)
decoder =AttnDecoderRnn(feature_size, hidden_size, 
                         len(vocab), num_layers)

# Load the trained model parameters
encoder.load_state_dict(torch.load(encoder_path))
decoder.load_state_dict(torch.load(decoder_path))

# If use gpu
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

    

In [20]:
temp = torch.rand(1,1,5)

In [27]:
import torch.nn.functional as F 
temp2 = F.softmax(Variable(temp),0)

TypeError: softmax() takes 1 positional argument but 2 were given

In [22]:
temp2[0]

Variable containing:
 1  1  1  1  1
[torch.FloatTensor of size 1x5]

In [16]:
torch.sum(temp2[0],1)

Variable containing:
1.00000e-02 *
  8.5966
[torch.FloatTensor of size 1x1]

In [21]:
temp[0]


 0.0029  0.9413  0.3740  0.9971  0.1829
[torch.FloatTensor of size 1x5]

In [5]:
temp = decoder.embed

In [8]:
x = Variable(torch.zeros(1,1,256)).cuda()
#temp = decoder.embed(x)

In [9]:
a = decoder.embed(x)

AssertionError: 